In [1]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [2]:
# --- QUANTUM LOOM: LAGRANGE RESONANCE (SYMMETRIC SCAN) ---
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import SamplerV2
import numpy as np

# 1. SETUP & CALIBRATION
# We define the Sun's "Gravity Well" depth.
# We tune it such that it exactly cancels Jupiter's pull at 60 degrees.
# This is the "Resonance Condition" for the Trojan Points.
SUN_TENSION = np.pi / 3  # ~1.047 radians
angles = np.linspace(0, 360, 36)

print(f"Calibrated Sun Tension: {SUN_TENSION:.3f} rad")

circuits = []
for deg in angles:
    qr = QuantumRegister(3, 'solar') # Q0=Sun, Q1=Jupiter, Q2=Asteroid
    cr = ClassicalRegister(3, 'meas')
    qc = QuantumCircuit(qr, cr)
    
    # A. INITIALIZE SUPERFLUID VACUUM
    # Q2 (The Asteroid location) starts in superposition
    qc.h(2) 
    
    # B. APPLY SUN'S GRAVITY (Fixed Anchor)
    # Negative rotation (The Well)
    qc.rzz(-SUN_TENSION, 0, 2)
    
    # C. APPLY JUPITER'S GRAVITY (Variable Wave)
    # FIX: GRAVITATIONAL SYMMETRY
    # Gravity depends on distance. L4 (60) and L5 (300) are equidistant from Jupiter.
    # We must use the "Shortest Arc" to Jupiter (at 0 degrees).
    
    if deg > 180:
        symmetric_deg = 360 - deg # e.g., 300 -> 60
    else:
        symmetric_deg = deg
        
    theta_pull = np.deg2rad(symmetric_deg)
    
    # Positive rotation (The Hill/Wave)
    # At 60 deg, theta_pull = pi/3. 
    # Net Phase = -pi/3 (Sun) + pi/3 (Jup) = 0.
    qc.rzz(theta_pull, 1, 2)
    
    # D. MEASURE INTERFERENCE
    # Destructive Interference (Phase 0) -> High Probability of '0'
    # Constructive Interference (Phase High) -> High Entropy / Noise
    qc.h(2) 
    qc.measure(2, 2) # We only really care about Q2
    
    circuits.append(qc)

# 2. RUN
print(f"Scanning Solar System for Lagrange Points (0-360 deg)...")
isa_circuits = transpile(circuits, backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=1024)
print(f"Job ID: {job.job_id()}")

result = job.result()

# 3. ANALYZE & VISUALIZE
print("\n=== SOLAR SYSTEM RESONANCE MAP ===")
print(f"{'ANGLE':<8} | {'STABILITY':<10} | {'LOCATION':<15} | {'VISUAL'}")
print("-" * 60)

for i, deg in enumerate(angles):
    # Get counts for Q2 measurement
    counts = result[i].data.meas.get_counts()
    total = sum(counts.values())
    
    # We look for '0' in the MSB (Bit 2).
    # Bitstring format: "b2 b1 b0"
    stable_shots = 0
    for key, val in counts.items():
        # Remove spaces
        k = key.replace(" ", "")
        # Check Bit 2 (Index 0 in string)
        if k[0] == '0':
            stable_shots += val
            
    stability = stable_shots / total
    
    # Identify Zones
    label = ""
    if 50 <= deg <= 70: label = "L4 (GREEKS)"
    elif 290 <= deg <= 310: label = "L5 (TROJANS)"
    elif deg == 0 or deg == 360: label = "JUPITER"
    elif deg == 180: label = "L3"
    
    # ASCII Graph
    bar_len = int((stability - 0.5) * 40) # Scale 50-100% to 0-20 chars
    if bar_len < 0: bar_len = 0
    bar = "█" * bar_len
    
    print(f"{deg:<8.0f} | {stability*100:.1f}%      | {label:<15} | {bar}")

print("-" * 60)

Calibrated Sun Tension: 1.047 rad
Scanning Solar System for Lagrange Points (0-360 deg)...
Job ID: d686g4re4kfs73d2nl60

=== SOLAR SYSTEM RESONANCE MAP ===
ANGLE    | STABILITY  | LOCATION        | VISUAL
------------------------------------------------------------
0        | 71.6%      | JUPITER         | ████████
10       | 81.1%      |                 | ████████████
21       | 86.6%      |                 | ██████████████
31       | 92.7%      |                 | █████████████████
41       | 96.3%      |                 | ██████████████████
51       | 97.9%      | L4 (GREEKS)     | ███████████████████
62       | 98.1%      | L4 (GREEKS)     | ███████████████████
72       | 97.4%      |                 | ██████████████████
82       | 94.1%      |                 | █████████████████
93       | 88.9%      |                 | ███████████████
103      | 84.5%      |                 | █████████████
113      | 75.5%      |                 | ██████████
123      | 69.7%      |               